In [5]:
import pickle
import numpy as np
import tensorflow as tf

# Load pickled data
with open('small_traffic_set/small_train_traffic.p', mode='rb') as f:
    data = pickle.load(f)  

In [6]:
# split data
X_train, y_train= data['features'], data['labels']

In [36]:
# Setup Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten,Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D

In [8]:
# Build from the previous network.

# model = Sequential()
# model.add(Flatten(input_shape=(32, 32, 3)))
# model.add(Dense(128))
# model.add(Activation('relu'))
# model.add(Dense(5))
# model.add(Activation('softmax'))
# model.summary()

# Add a convolutional layer with 32 filters, a 3x3 kernel, and valid padding before the flatten layer.
# Add a ReLU activation after the convolutional layer.
# Train for 3 epochs again, should be able to get over 50% accuracy.

In [37]:
# TODO: Build Convolutional Neural Network in Keras Here
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu',
                 input_shape=(32, 32, 3)))
# Add a 2x2 max pooling layer immediately following your convolutional layer.
model.add(MaxPooling2D(2,2))
# Add a dropout layer after the pooling layer. Set the dropout rate to 50%.
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               921728    
_________________________________________________________________
activation_15 (Activation)   (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 645       
__________

In [39]:
# Preprocess data
X_normalized = np.array(X_train / 255.0 - 0.5 )

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

In [54]:
# compile and train model
# Training for 3 epochs should result in > 50% accuracy
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, epochs=8, validation_split=0.2)

Train on 80 samples, validate on 20 samples
Epoch 1/8
80/80 [==============================] - 0s 2ms/step - loss: 0.1903 - acc: 0.9250 - val_loss: 0.1574 - val_acc: 0.8500
Epoch 2/8
80/80 [==============================] - 0s 732us/step - loss: 0.1499 - acc: 0.9000 - val_loss: 0.0788 - val_acc: 1.0000
Epoch 3/8
80/80 [==============================] - 0s 720us/step - loss: 0.1567 - acc: 0.9125 - val_loss: 0.0551 - val_acc: 1.0000
Epoch 4/8
80/80 [==============================] - 0s 758us/step - loss: 0.0851 - acc: 1.0000 - val_loss: 0.0805 - val_acc: 1.0000
Epoch 5/8
80/80 [==============================] - 0s 655us/step - loss: 0.0911 - acc: 0.9750 - val_loss: 0.0424 - val_acc: 1.0000
Epoch 6/8
80/80 [==============================] - 0s 703us/step - loss: 0.0581 - acc: 1.0000 - val_loss: 0.0463 - val_acc: 1.0000
Epoch 7/8
80/80 [==============================] - 0s 751us/step - loss: 0.0595 - acc: 1.0000 - val_loss: 0.0308 - val_acc: 1.0000
Epoch 8/8
80/80 [========================

In [45]:
# Try to get the highest validation accuracy possible. Feel free to use all the previous concepts and train for as many epochs as needed.
# Select your best model and train it one more time.
# Use the test data and the Keras evaluate() method to see how well the model does.

In [55]:
# evaluate model against the test data
with open('small_traffic_set/small_test_traffic.p', 'rb') as f:
    data_test = pickle.load(f)

X_test = data_test['features']
y_test = data_test['labels']

# preprocess data
X_normalized_test = np.array(X_test / 255.0 - 0.5 )
y_one_hot_test = label_binarizer.fit_transform(y_test)

print("Testing")

metrics = model.evaluate(X_normalized_test, y_one_hot_test)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))  

Testing
20/20 [==============================] - 0s 216us/step
loss: 0.05307658761739731
acc: 1.0


In [56]:
import os
import sys
import cv2
import json
import numpy as np
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax
    
def get_result(metrics):
    
    result = {'is_correct': False, 'error': False, 'values': [],
              'output': '', 'feedback': '', 'comment': ""}
    
    try:        
        
        if metrics is not Ellipsis:
            metric_value = metrics[1]

            if metric_value < 0.5:
                result["is_correct"] = False
                result["comment"] = 'I bet you can do better than 50%'
                result["feedback"] = 'Accuracy was '+ str(metric_value)
            elif metric_value < 0.90:
                result["is_correct"] = True
                result["comment"] = 'But can you get above 90%?'
                result["feedback"] = 'Accuracy was '+ str(metric_value)
            else:
                result["is_correct"] = True
                result["comment"] = 'Good Job, accuracy was above 90%'
                result["feedback"] = 'Nice, accuracy was '+ str(metric_value)
        else:
            result["is_correct"] = False
            result["comment"] = 'You still need to evaluate the test data'
            result["feedback"] = 'evaluate method was empty'
            
    except Exception as err:
        result['is_correct'] = False
        result['feedback'] = 'Oops looks like you got an Error'
        result['error'] = str(err)
    
    return result

def run_grader(metrics):
    
    try:
    # Get grade result information
        result = get_result(metrics)
    except Exception as err:
        # Default error result
        result = {
            'correct': False,
            'feedback': 'Something went wrong with your submission:',
            'comment': str(err)}

    feedback = result.get('feedback')
    comment = result.get('comment')

    print(feedback,comment)



In [57]:
### DON'T MODIFY ANYTHING BELOW ###
### Be sure to run all cells above before running this cell ###

try:
    run_grader(metrics)
except Exception as err:
    print(str(err))

Nice, accuracy was 1.0 Good Job, accuracy was above 90%
